In [ ]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from tqdm.notebook import tqdm
import time
import string
import os
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import codecs
import pickle
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
import annoy
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer

DATA_PATH = 'chatbot_files/'

tqdm.pandas()  # enable 'progress_apply' function

tbeg = time.perf_counter()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Предобработаем ответы mail.ru из файла: к каждому вопросу присоединим 1 ответ и запишем в файл на будущее.
# Это позволит нам сэкономить время и ресурсы при дальнейшем препроцессинге текста

question = None
written = False

# Мы идем по всем записям, берем первую строку как вопрос
# и после знака --- находим ответ
with codecs.open(DATA_PATH + "prepared_answers.txt", "w", "utf_8_sig") as fout:
    with open("Otvety.txt", "r") as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [ ]:
# Так как строки иногда разрываются (по символу \n) не после последнего поля), то приходится
# преобразовать файл 'ProductsDataset.csv' так, чтобы строка заканчивалась после последовательности ".jpg"
# В итоге получаем новый файл 'prodDS.csv'.


out_line = ''
c = 0
with codecs.open(DATA_PATH + "prodDS.csv", "w", "utf_8_sig") as fout:
    with codecs.open("ProductsDataset.csv", "r", "utf_8_sig") as fin:
        for line in tqdm(fin):
            c += 1
            # print(c)
            # if c>10:
            #   break
            if c > 1:
                if line.endswith(".jpg\n") != True:
                    # print('No')
                    # print(line)
                    not_EOL = True
                    out_line = (out_line+line).replace('\n', ' ')
                    continue
                else:
                    # print('Yes')
                    out_line = out_line+line
                    # print(out_line)
                    fout.write(out_line)
                    not_EOL = False
                    out_line = ''
            else:
                fout.write(line)

In [ ]:
df = pd.read_csv(DATA_PATH + "prodDS.csv")
df = df.fillna('')
df['label'] = 0  # Добавим разметку
df.columns = ['title', 'description', 'product_id', 'category_id', 'subcategory_id',
              'properties', 'image_links', 'label']  # исправим 'descrirption' -> 'description'

In [ ]:
# Объединим названия и описания в один текстовый столбец так, как:
# "...продуктовым можно считать запрос, который равен названию или описанию товара" .

df1 = df[['description', 'label']]
df2 = df[['title', 'label']]
df1.columns = ['text', 'label']
df2.columns = ['text', 'label']

df_products = pd.concat([df1, df2], ignore_index=True)

In [ ]:
# Соберем датафрейм с вопросами из болталки
questions = []
with codecs.open(DATA_PATH + "prepared_answers.txt", "r", "utf_8_sig") as f:
    for line in tqdm(f):
        spls = line.split("\t")
        questions.append(spls[0])

df_questions = pd.DataFrame({'text': questions})
df_questions['label'] = 1  # Добавим разметку

In [ ]:
# Собираем итоговый датасет
table = pd.concat([df_products, df_questions], ignore_index=True)
# Remove rows with empty text cells
table.drop(index=table[table.text == ''].index, inplace=True)
table.reset_index(drop=True, inplace=True)

In [ ]:
r = RussianStemmer()

sw = set(get_stop_words("ru")).union(
    set('1234567890qwertyuiopasdfghjklzxcvbnmйцукенгшщзхъфываепнроглшщджэячсмитьбю'))
exclude = set(string.punctuation)


def pprocess(line):
    line = re.sub("[\x0b\x0c\s]", ' ', line.lower())
    line = re.sub("(б\.у)|(б\/у)|(б\\у)", 'бу', line)
    line = re.sub("[^а-яё \d\w]", ' ', line)

    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [r.stem(i) for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return ' '.join(spls)

In [ ]:
t = time.perf_counter()
table_processed = table['text'].progress_apply(lambda x: pprocess(x))
print('Elapsed, s:', time.perf_counter() - t)

In [ ]:
X = table_processed.to_list()
y = table['label'].to_list()

plt.hist(y)
plt.show()
s1 = (table['label'] == 1).sum()
s0 = (table['label'] == 0).sum()
print(f'Product({s0})/Non-product({s1}) = {s0/s1}')
# Классы распределены неравномерно - будем использовать stratify при разбиении на тест и трейн

In [ ]:
# Make datasets with similar classes distribution
TX_train, TX_test, ly_train, ly_test, text_train, text_test = train_test_split(
    X, y, table['text'], test_size=0.20, random_state=42, shuffle=True, stratify=y)

In [ ]:
t = time.perf_counter()
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=50000)
vectorizer.fit(TX_train)
print('Elapsed, s:', time.perf_counter() - t)

In [ ]:
t = time.perf_counter()

X_train = vectorizer.transform(TX_train)
y_train = np.array(ly_train)

X_test = vectorizer.transform(TX_test)
y_test = np.array(ly_test)

print('Elapsed, s:', time.perf_counter() - t)

In [ ]:
X_train

In [ ]:
# create LinearSVM model object and fit the model
model_svm = svm.LinearSVC()
model_svm.fit(X_train, y_train)

In [ ]:
# Сохраняем классификатор
with open(DATA_PATH + 'query_model_stem_tdfv_best.pickle', 'wb') as f:
    pickle.dump(model_svm, f)

In [ ]:
# Посмотрим на различные рекомендуемые метрики при дисбалансе классов
predictions = model_svm.predict(X_test)
f1_s = f1_score(y_test, predictions)
f2 = fbeta_score(y_test, predictions, beta=2.0)
accuracy = (predictions == y_test).mean()
print(accuracy, f1_s, f2)

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = y_test
y_pred = predictions
cm = confusion_matrix(y_true, y_pred)
print('Всего запросов в тестовой выборке:', len(y_test))
print('Правильно классифицированных продуктовых запросов:',
      cm[0, 0], 'Ошибочно потерянных продуктовых запросов:', cm[0, 1])
print('Ошибочно принятых за продуктовые запросы:',
      cm[1, 0], 'Правильно классифицированных мусорных запросов:', cm[1, 1])
print('Процент потерянных истинных продуктовых запросов:',
      100*cm[0, 1]/cm[0, 0])
print('Процент принятых мусорных запросов:', 100*cm[1, 0]/cm[1, 1])

In [ ]:
sw = set(get_stop_words("ru")).union(
    set('1234567890qwertyuiopasdfghjklzxcvbnmйцукенгшщзхъфываепнроглшщджэячсмитьбю'))
exclude = set(string.punctuation)
morpher = MorphAnalyzer()


def products_preprocess_txt(line):
    line = re.sub("[\x0b\x0c\s]", ' ', line.lower())
    line = re.sub("(б\.у)|(б\/у)|(б\\у)", 'бу', line)
    line = re.sub("[^а-яё \d\w]", ' ', line)

    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
# Для обучения Word2Vec создадим датасет из названий и описаний длиной больше 20 символов.
df_products = pd.concat(
    [df1.loc[df1.text.str.len() > 20], df2], ignore_index=True)

In [ ]:
t = time.perf_counter()
products_processed = df_products['text'].progress_apply(
    lambda x: products_preprocess_txt(x))
print('Elapsed, s:', time.perf_counter() - t)

In [ ]:
# Обучаем модель для продуктовых запросов
t = time.perf_counter()
model_w2v_products = Word2Vec(
    sentences=products_processed.to_list(), vector_size=100, min_count=1, window=5)
print('Elapsed, s:', time.perf_counter() - t)

In [ ]:
# Складываем в индекс все наши названия продуктовых запросов
t = time.perf_counter()
index = annoy.AnnoyIndex(100, 'angular')

index_map = {}
counter = 0


def annoy_indexing(row):
    global counter
    n_w2v = 0
    index_map[counter] = row['product_id'] + ' ' + row['title']
    product_query = products_preprocess_txt(row['title'])
    vector = np.zeros(100)
    for word in product_query:
        if word in model_w2v_products.wv:
            vector += model_w2v_products.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    index.add_item(counter, vector)
    counter += 1

df.apply(lambda x: annoy_indexing(x), 1)
index.build(10)

print('Elapsed, s:', time.perf_counter() - t)

In [ ]:
def find_description(product_title):
    preprocessed_title = products_preprocess_txt(product_title)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_title:
        if word in model_w2v_products.wv:
            vector += model_w2v_products.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    return index_map[answer_index[0]]

In [ ]:
find_description('Юбка детская ORBY')

In [ ]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)


def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
t = time.perf_counter()
sentences = []
c = 0
with codecs.open("Otvety.txt", "r", "utf_8_sig") as fin:
    for line in tqdm(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

print('Elapsed, s:', time.perf_counter() - t)

In [ ]:
sentences = [i for i in sentences if len(i) > 2]

In [ ]:
# Обучим модель word2vec на наших вопросах
t = time.perf_counter()
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)

print('Elapsed, s:', time.perf_counter() - t)

In [ ]:
# Складываем в индекс все наши вопросы из болталки
t = time.perf_counter()
index_s = annoy.AnnoyIndex(100, 'angular')

index_map_s = {}
counter = 0

with codecs.open(DATA_PATH + "prepared_answers.txt", "r", "utf_8_sig") as f:

    for line in tqdm(f):
        n_w2v = 0
        spls = line.split("\t")
        if len(spls) > 1:
            index_map_s[counter] = spls[1]
            question = preprocess_txt(spls[0])
            vector = np.zeros(100)
            for word in question:
                if word in model.wv:
                    vector += model.wv[word]
                    n_w2v += 1
            if n_w2v > 0:
                vector = vector / n_w2v
            index_s.add_item(counter, vector)

        counter += 1
index_s.build(10)
print('Elapsed, s:', time.perf_counter() - t)

In [ ]:
def find_answer(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index_s.get_nns_by_vector(vector, 1)
    return index_map_s[answer_index[0]]

In [ ]:
find_answer('Эбу в двенашке называется Итэлма что за эбу?')

In [ ]:
print('Total Run Time Elapsed, s:', time.perf_counter() - tbeg)

In [ ]:
try:
    # Подгружаем ранее сохраненный и обученный векторайзер
    with open(DATA_PATH + 'TfidfVectorizer.pickle', 'rb') as f:
        vectorizer = pickle.load(f)
    # Подгружаем ранее сохраненный и обученный классификатор
    with open(DATA_PATH + 'query_model_stem_tdfv_best.pickle', 'rb') as f:
        model_svm = pickle.load(f)

    # Подгружаем предварительно обученную модель продуктовых названий
    model_w2v_products = Word2Vec.load(DATA_PATH + "w2v_model_products")

    # Подгружаем предварительно подготовленный и сохранненый индекс продуктовых названий
    with open(DATA_PATH + 'index_map_products.pickle', 'rb') as f:
        index_map = pickle.load(f)
    index = annoy.AnnoyIndex(100, 'angular')
    index.load(DATA_PATH + 'products.ann')

    # Подгружаем предварительно обученную модель вопросов из болталки
    model = Word2Vec.load(DATA_PATH + "w2v_model")

    # Подгружаем предварительно подготовленный и сохранненый индекс ответов из болталки
    with open(DATA_PATH + 'index_map_speaker.pickle', 'rb') as f:
        index_map_s = pickle.load(f)
    index_s = annoy.AnnoyIndex(100, 'angular')
    index_s.load(DATA_PATH + 'speaker.ann')
except:
    pass


r = RussianStemmer()
morpher = MorphAnalyzer()
sw_ = set(get_stop_words("ru"))
sw = sw_.union(
    set('1234567890qwertyuiopasdfghjklzxcvbnmйцукенгшщзхъфываепнроглшщджэячсмитьбю'))
exclude = set(string.punctuation)

def get_answer(text):

    def pprocess(line):
        # используется для обработки текстовых данных при обучении классификатора
        line = re.sub("[\x0b\x0c\s]", ' ', line.lower())
        line = re.sub("(б\.у)|(б\/у)|(б\\у)", 'бу', line)
        line = re.sub("[^а-яё \d\w]", ' ', line)
        spls = "".join(i for i in line.strip() if i not in exclude).split()
        spls = [r.stem(i) for i in spls]
        spls = [i for i in spls if i not in sw and i != ""]
        return ' '.join(spls)

    def products_preprocess_txt(line):
        # используется для обработки текстовых данных при сворачивании продуктовых названий в  Word2Vec
        line = re.sub("[\x0b\x0c\s]", ' ', line.lower())
        line = re.sub("(б\.у)|(б\/у)|(б\\у)", 'бу', line)
        line = re.sub("[^а-яё \d\w]", ' ', line)
        spls = "".join(i for i in line.strip() if i not in exclude).split()
        spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
        spls = [i for i in spls if i not in sw and i != ""]
        return spls

    def preprocess_txt(line):
        # используется для обработки текстовых данных при сворачивании вопросов из болталки в  Word2Vec
        spls = "".join(i for i in line.strip() if i not in exclude).split()
        spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
        spls = [i for i in spls if i not in sw_ and i != ""]
        return spls

    def find_description(product_title):
        # находит id и title входному продуктовому запросу
        preprocessed_title = products_preprocess_txt(product_title)
        n_w2v = 0
        vector = np.zeros(100)
        for word in preprocessed_title:
            if word in model_w2v_products.wv:
                vector += model_w2v_products.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        answer_index = index.get_nns_by_vector(vector, 1)
        return index_map[answer_index[0]]

    def find_answer(question):
        # находит ответ из болталки по  входному вопросу
        preprocessed_question = preprocess_txt(question)
        n_w2v = 0
        vector = np.zeros(100)
        for word in preprocessed_question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        answer_index = index_s.get_nns_by_vector(vector, 1)
        return index_map_s[answer_index[0]]



    # product or speaker?
    if model_svm.predict(vectorizer.transform([pprocess(text)]))[0] == 1:
        # speaker
        return find_answer(text)
    else:
        # product
        return find_description(text)

In [ ]:
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Привет! Назови свой вопрос')

def text_msg(update: Update, context: CallbackContext):
    txt_q = update.message.text # текст который пришёл
    update.message.reply_text(get_answer(txt_q))

In [ ]:
updater = Updater("5987399402:AAFBjEvfZaGBcCpN3Csruh99Gr368kTP5E8", use_context=True) # Token
dispatcher = updater.dispatcher

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, text_msg))

# Start the Bot
updater.start_polling()
updater.idle()